# **Ministry Budget: Regression**

A partir de nuestros datos limpios vamos a proceder con realizar una regresión no lineal. Elegimos esta pues en nuestra análisis y exploración de datos notamos que no existe correlación entre variables, pues el coeficiente es muy bajo, esto quiere decir que no existe constante que mantenga la proporción, algo común considerando que mediana y promedio difieren.

---
## **Libraries**

In [3]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;

from sklearn.model_selection import train_test_split;
from sklearn.preprocessing import StandardScaler;

---
## **No-linear Regression**

In [12]:
class NonlinearRegression:
    def __init__(self,  poly_degree, X):
        self.poly_degree = poly_degree;
        xshape1,xshape2 = X.shape
        self.weights = np.random.rand(xshape1,xshape2);
        self.bias = np.random.random();

    def poly_features(self, X):
        n_samples, n_features = X.shape;
        features_vector = np.ones((n_samples, 1));

        for degree in range(1, self.poly_degree + 1):
            for feature in range(n_features):
                new_feature = X[:, feature] ** degree;
                features_vector = np.hstack((features_vector, new_feature.reshape(n_samples, 1)));
        
        self.weights = np.random.rand(features_vector.shape[1])
        return features_vector;

    def predict(self, X):
        X_poly = self.poly_features(X);
        return np.dot(X_poly, self.weights) + self.bias;

    def loss_function(self, X, Y, poly_lambda, reg_type='L2'):
        y_pred = self.predict(X);
        base_loss = np.mean((Y - y_pred) ** 2) / 2;

        if reg_type == 'L1':
            regularization = poly_lambda * np.sum(np.abs(self.weights));
        else:
            regularization = poly_lambda * np.sum(self.weights ** 2);

        return base_loss + regularization;

    def gradients(self, X, Y, poly_lambda):
        n = len(Y)
        X_poly = self.poly_features(X)
        y_pred = self.predict(X)
        error = Y - y_pred
        dw = -(np.dot(X_poly.T, error) / n) + 2 * poly_lambda * self.weights
        db = -np.sum(error) / n
        return dw, db

    def update_params(self, dw, db, poly_alpha):
        self.weights -= poly_alpha * dw
        self.bias -= poly_alpha * db

    def train(self, X, Y, poly_alpha, poly_epochs, poly_lambda):
        for epoch in range(poly_epochs):
            dw, db = self.gradients(X, Y, poly_lambda)
            self.update_params(dw, db, poly_alpha)

            if epoch % 1000 == 0:
                loss = self.loss_function(X, Y, poly_lambda)
                print(f'Epoch {epoch}, Loss: {loss}')

Ahora en este punto ya contamos con el modelo no lineal, lo que nos quedaría realizar es la parte del entrenamiento y testeo de datos, de tal forma que podamos iterar hasta llegar al mínimo error. Para ello continuamos de la siguiente manera:

In [5]:
np.random.seed(11);

train_data = pd.read_csv('./db/trainDataModified.csv');

x_data = train_data.drop(['MTO_PIA'], axis=1);
y_data = train_data['MTO_PIA'];

print(x_data.info());
print(x_data.head(5));

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   PROGRAMA_PPTAL   1464 non-null   int64
 1   ACT_OBRA_ACCINV  1464 non-null   int64
 2   GRUPO_FN         1464 non-null   int64
 3   META             1464 non-null   int64
 4   AVAN_FISICO_SEM  1464 non-null   int64
 5   DISTRITO         1464 non-null   int64
 6   GENERICA         1464 non-null   int64
 7   SUBGENERICA      1464 non-null   int64
 8   SUBGENERICA_DET  1464 non-null   int64
 9   ESPECIFICA       1464 non-null   int64
 10  ESPECIFICA_DET   1464 non-null   int64
dtypes: int64(11)
memory usage: 125.9 KB
None
   PROGRAMA_PPTAL  ACT_OBRA_ACCINV  GRUPO_FN  META  AVAN_FISICO_SEM  DISTRITO  \
0               1                2         2     6                6        17   
1               1                2         2    20                6        17   
2               2               

Una vez que hemos cargado nuestra data y hemos apartado la columna MTO_PIA, es hora de crear nuestra data de test y de entrenamiento, para ello tomamos un 20% para entrenamiento. Por otro lado, vamos a usar normalizar nuestros datos utilizando la función StandardScaler de la librería de Sklearn.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=11)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train);
x_test_scaled = scaler.transform(x_test);

print(x_train.sort_values(by='GRUPO_FN'));
print(x_train_scaled);

      PROGRAMA_PPTAL  ACT_OBRA_ACCINV  GRUPO_FN  META  AVAN_FISICO_SEM  \
240                1                0         0     1                6   
1138               1                0         0     1                6   
1394               1                0         0     1                6   
286                1                0         0     1                6   
1282               1                0         0     1                6   
...              ...              ...       ...   ...              ...   
533                2                6         5    26                0   
976                2                5         5    17                0   
638                2                6         5    12               51   
1341               2                6         5    24                4   
1104               2                6         5    25                4   

      DISTRITO  GENERICA  SUBGENERICA  SUBGENERICA_DET  ESPECIFICA  \
240         13         0            1    

Ahora es momento de poner a prueba nuestro modelo, para ello tenemos un diccionario con los parámetros que se le asignarán a nuestro modelo, fácil de actualizar. No obstante invocamos nuestro modelo de entrenamiento y además buscamos las variables predictoras tanto para train como para test.

In [14]:
parametersModel = {
  'poly_degree': 2,
  'poly_alpha': 0.0001,
  'poly_epochs': 100000,
  'poly_lambda': 0.01,
};

NR = NonlinearRegression(poly_degree=parametersModel['poly_degree'], X=x_train_scaled)
NR.train(x_train_scaled, y_train.to_numpy(), poly_alpha=parametersModel['poly_alpha'], poly_epochs=parametersModel['poly_epochs'], poly_lambda=parametersModel['poly_lambda']);

y_pred_train = NR.predict(x_train_scaled)
y_pred_test = NR.predict(x_test_scaled)

Epoch 0, Loss: 18689520272.68854
Epoch 100, Loss: 18489716420.31615
Epoch 200, Loss: 18326152950.674778
Epoch 300, Loss: 18192187988.640736
Epoch 400, Loss: 18082630101.93538
Epoch 500, Loss: 17992890730.74262
Epoch 600, Loss: 17919365503.808666
Epoch 700, Loss: 17859250174.817715
Epoch 800, Loss: 17809991533.0637
Epoch 900, Loss: 17769718408.446003
Epoch 1000, Loss: 17736734159.862057
Epoch 1100, Loss: 17709701014.79629
Epoch 1200, Loss: 17687574105.607677
Epoch 1300, Loss: 17669485572.166264
Epoch 1400, Loss: 17654637778.90388
Epoch 1500, Loss: 17642539145.79811
Epoch 1600, Loss: 17632592744.282295
Epoch 1700, Loss: 17624507446.527233
Epoch 1800, Loss: 17617776649.40861
Epoch 1900, Loss: 17612371515.11124
Epoch 2000, Loss: 17607884327.52615
Epoch 2100, Loss: 17604231566.40974
Epoch 2200, Loss: 17601270511.149265
Epoch 2300, Loss: 17598771874.84303
Epoch 2400, Loss: 17596807224.07543
Epoch 2500, Loss: 17595173518.03724
Epoch 2600, Loss: 17593834507.453026
Epoch 2700, Loss: 17592711587

In [ ]:
feature_x_index = 0;
feature_y_index = 1;

fig = plt.figure(figsize=(12, 8))

# Gráfico para el conjunto de entrenamiento.
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.scatter(x_train_scaled[:, feature_x_index], x_train_scaled[:, feature_y_index], y_train, color='blue', label='Datos reales')
ax1.scatter(x_train_scaled[:, feature_x_index], x_train_scaled[:, feature_y_index], y_pred_train, color='red', label='Predicción del modelo')
ax1.set_title('Conjunto de Entrenamiento')
ax1.set_xlabel('Característica x')
ax1.set_ylabel('Característica y')
ax1.set_zlabel('Y (MTO_PIA)')
ax1.legend()

# Gráfico para el conjunto de prueba.
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.scatter(x_test_scaled[:, feature_x_index], x_test_scaled[:, feature_y_index], y_test, color='blue', label='Datos reales')
ax2.scatter(x_test_scaled[:, feature_x_index], x_test_scaled[:, feature_y_index], y_pred_test, color='red', label='Predicción del modelo')
ax2.set_title('Conjunto de Prueba')
ax2.set_xlabel('Característica x')
ax2.set_ylabel('Característica y')
ax2.set_zlabel('Y (MTO_PIA)')
ax2.legend()

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined